In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
def extract_scalars_from_events_file(file_path):
    # Initialize the EventAccumulator
    ea = event_accumulator.EventAccumulator(file_path)
    ea.Reload()

    # Get the list of all scalar tags
    scalar_tags = ea.Tags()["scalars"]

    # Extract scalar data
    data = {}
    for tag in scalar_tags:
        events = ea.Scalars(tag)
        steps = [event.step for event in events]
        values = [event.value for event in events]
        data[tag] = (steps, values)

    return data

In [ ]:
# Path to your .tfevents file
file_path = "/work/tc064/tc064/s2567498/virtual-sensing/lightning_logs/mamba/version_5/events.out.tfevents.1722416608.r2i7n8.276037.0"

# Extract and plot the data
scalar_data = extract_scalars_from_events_file(file_path)
plot_scalar_data(scalar_data)